In [6]:
import os
import sys
import datetime
import pandas as pd
import boto3
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, datediff, lit, date_format, count, when
from boto3.dynamodb.conditions import Key
from dotenv import load_dotenv
from tabulate import tabulate
from IPython.display import display, Markdown

# Configuração inicial otimizada
load_dotenv()

# Configurar Spark com otimizações e timeouts maiores
spark = SparkSession.builder \
    .appName("Abandono6meses") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.network.timeout", "600s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

# Reduzir logs para melhor performance
spark.sparkContext.setLogLevel("WARN")

dynamodb = boto3.resource('dynamodb', region_name='sa-east-1')
table = dynamodb.Table("Tabela-teste-tarefas")

# Data de referência
hoje = datetime.date.today()
print(f"=== ANÁLISE DE TAREFAS ABANDONADAS (OTIMIZADA) ===")
print(f"Data de referência: {hoje}")

# 1. GERAR TODAS AS DATAS DOS ÚLTIMOS 6 MESES
print(f"\n1. GERANDO DATAS DOS ÚLTIMOS 6 MESES...")

def gerar_datas_6_meses(data_atual):
    """Gera todas as datas dos últimos 6 meses"""
    datas = []
    data_inicio = data_atual - datetime.timedelta(days=180)
    data_iter = data_inicio
    while data_iter <= data_atual:
        datas.append(data_iter.strftime("%Y%m%d"))
        data_iter += datetime.timedelta(days=1)
    return datas

todas_datas = gerar_datas_6_meses(hoje)
print(f"Período de análise: {todas_datas[0]} até {todas_datas[-1]}")
print(f"Total de datas para verificar: {len(todas_datas)}")

# 2. BUSCAR DADOS COM OTIMIZAÇÃO
print(f"\n2. BUSCANDO DADOS COM QUERY OTIMIZADA...")

todos_registros = []
datas_com_dados = []
total_queries = 0
lote_size = 50

for i in range(0, len(todas_datas), lote_size):
    lote_datas = todas_datas[i:i+lote_size]
    print(f"Processando lote {i//lote_size + 1}: datas {lote_datas[0]} a {lote_datas[-1]}")
    
    for data in lote_datas:
        pk = f"LIST#{data}"
        total_queries += 1
        
        try:
            response = table.query(
                KeyConditionExpression=Key("PK").eq(pk),
                ProjectionExpression="PK, SK, #nm, task_type, #st, created_at, completed_at, #usr, user_ID",
                ExpressionAttributeNames={
                    "#nm": "name",
                    "#st": "status", 
                    "#usr": "user"
                }
            )
            registros = response.get("Items", [])
            
            if registros:
                todos_registros.extend(registros)
                datas_com_dados.append(data)
                
        except Exception as e:
            print(f"  Erro ao buscar {pk}: {e}")
    
    if len(todos_registros) > 0:
        print(f"  Progresso: {len(todos_registros)} registros encontrados até agora")

print(f"\nResultado da busca:")
print(f"- Total de queries executadas: {total_queries}")
print(f"- Datas com dados: {len(datas_com_dados)}")
print(f"- Total de registros encontrados: {len(todos_registros)}")

if datas_com_dados:
    print(f"- Primeira data com dados: {min(datas_com_dados)}")
    print(f"- Última data com dados: {max(datas_com_dados)}")

if not todos_registros:
    print("❌ Nenhum registro encontrado!")
    spark.stop()
    exit()

# 3. PROCESSAMENTO HÍBRIDO (Pandas primeiro para datasets pequenos)
print(f"\n3. PROCESSANDO DADOS...")

# Se poucos registros, usar Pandas (mais rápido)
if len(todos_registros) <= 100:
    print("📊 Usando processamento Pandas (dataset pequeno)")
    
    # Converter para pandas
    df_pandas = pd.DataFrame(todos_registros)
    print("Colunas disponíveis:", list(df_pandas.columns))
    print(f"Total de registros: {len(df_pandas)}")
    
    # Mostrar alguns exemplos
    print("\nPrimeiros 3 registros:")
    print(df_pandas[['name', 'task_type', 'status', 'created_at']].head(3).to_string(index=False))
    
    # 4. VERIFICAR DISTRIBUIÇÕES COM PANDAS
    print(f"\n4. VERIFICANDO DISTRIBUIÇÕES...")
    
    print("Distribuição de status:")
    status_counts = df_pandas['status'].value_counts()
    for status, count in status_counts.items():
        print(f"  {status}: {count}")
    
    print("\nDistribuição de tipos de tarefa:")
    tipo_counts = df_pandas['task_type'].value_counts()
    for tipo, count in tipo_counts.items():
        print(f"  {tipo}: {count}")
    
    # 5. FILTRAR TAREFAS EM ABERTO
    print(f"\n5. FILTRANDO TAREFAS EM ABERTO...")
    df_em_aberto = df_pandas[df_pandas['status'] == 'todo'].copy()
    count_em_aberto = len(df_em_aberto)
    print(f"Tarefas com status 'todo': {count_em_aberto}")
    
    if count_em_aberto == 0:
        print("❌ Nenhuma tarefa em aberto encontrada!")
        print("Status disponíveis:", df_pandas['status'].unique().tolist())
        spark.stop()
        exit()
    
    # 6. CALCULAR DIAS EM ABERTO COM PANDAS
    print(f"\n6. CALCULANDO DIAS EM ABERTO...")
    
    # Converter data para datetime
    df_em_aberto['created_at'] = pd.to_datetime(df_em_aberto['created_at'])
    hoje_pd = pd.Timestamp(hoje)
    df_em_aberto['dias_aberto'] = (hoje_pd - df_em_aberto['created_at']).dt.days
    
    print("Todas as tarefas em aberto com dias calculados:")
    tarefas_display = df_em_aberto[['name', 'task_type', 'created_at', 'dias_aberto']].sort_values('dias_aberto', ascending=False)
    print(tarefas_display.to_string(index=False))
    
    # 7. APLICAR CRITÉRIOS COM PANDAS
    print(f"\n7. APLICANDO CRITÉRIOS DE ABANDONO...")
    print("Critérios de abandono:")
    print("- Tarefas normais (Tarefa a Ser Feita): > 15 dias")
    print("- Itens de compra (Item de Compra): > 30 dias")
    
    # Separar por tipo
    tarefas_normais = df_em_aberto[df_em_aberto['task_type'] == 'Tarefa a Ser Feita']
    itens_compra = df_em_aberto[df_em_aberto['task_type'] == 'Item de Compra']
    
    print(f"\nAnálise por tipo:")
    print(f"- Tarefas normais em aberto: {len(tarefas_normais)}")
    print(f"- Itens de compra em aberto: {len(itens_compra)}")
    
    # Aplicar critérios
    tarefas_abandonadas_normais = tarefas_normais[tarefas_normais['dias_aberto'] > 15]
    itens_abandonados = itens_compra[itens_compra['dias_aberto'] > 30]
    
    count_abandonadas_normais = len(tarefas_abandonadas_normais)
    count_itens_abandonados = len(itens_abandonados)
    
    print(f"\nApós aplicar critérios:")
    print(f"- Tarefas normais abandonadas (>15 dias): {count_abandonadas_normais}")
    print(f"- Itens de compra abandonados (>30 dias): {count_itens_abandonados}")
    
    # Combinar abandonadas
    df_abandonadas = pd.concat([tarefas_abandonadas_normais, itens_abandonados], ignore_index=True)
    count_abandonadas = len(df_abandonadas)
    
    print(f"\n🎯 RESULTADO PRINCIPAL: {count_abandonadas} TAREFAS ABANDONADAS ENCONTRADAS!")
    
    # 8. MOSTRAR DETALHES
    print(f"\n8. DETALHES DAS TAREFAS ABANDONADAS...")
    
    if count_abandonadas > 0:
        print("Tarefas abandonadas (ordenadas por dias em aberto):")
        abandonadas_display = df_abandonadas[['name', 'task_type', 'created_at', 'dias_aberto']].sort_values('dias_aberto', ascending=False)
        print(abandonadas_display.to_string(index=False))
        
        # TABELA RESUMO
        print(f"\n=== TABELA RESUMO - TAREFAS ABANDONADAS ===")
        df_abandonadas['mes_criacao'] = df_abandonadas['created_at'].dt.strftime('%Y-%m')
        
        # Criar tabela resumo pivot
        resumo = df_abandonadas.groupby(['mes_criacao', 'task_type']).size().reset_index(name='count')
        resumo_pivot = resumo.pivot(index='mes_criacao', columns='task_type', values='count').fillna(0).astype(int)
        
        print("Tarefas abandonadas por mês e tipo:")
        print(resumo_pivot.to_string())
        
    else:
        print("❌ Nenhuma tarefa abandonada encontrada com os critérios aplicados!")
        
        # Debug info
        print("\n📊 Detalhamento das tarefas em aberto (para debug):")
        debug_info = df_em_aberto.groupby('task_type').agg({
            'name': 'count',
            'dias_aberto': 'max'
        }).rename(columns={'name': 'total'})
        print(debug_info.to_string())
        
        print("\nAmostra das tarefas em aberto:")
        amostra = df_em_aberto[['name', 'task_type', 'dias_aberto']].sort_values('dias_aberto', ascending=False).head(10)
        print(amostra.to_string(index=False))

else:
    # USAR SPARK PARA DATASETS MAIORES
    print("⚙️ Usando processamento Spark (dataset grande)")
    
    # Criar DataFrame Spark
    df_completo = spark.createDataFrame(todos_registros)
    
    print("Colunas disponíveis:", df_completo.columns)
    print(f"Total de registros: {df_completo.count()}")
    
    print("\nPrimeiros registros:")
    df_completo.select("name", "task_type", "status", "created_at").show(3, truncate=False)
    
    # Continuar com processamento Spark...
    print("Distribuição de status:")
    df_completo.groupBy("status").count().show()
    
    print("Distribuição de tipos de tarefa:")
    df_completo.groupBy("task_type").count().show()
    
    # Filtrar e processar com Spark
    df_em_aberto = df_completo.filter(col("status") == "todo")
    count_em_aberto = df_em_aberto.count()
    
    if count_em_aberto > 0:
        df_em_aberto = df_em_aberto.withColumn("created_at", to_date(col("created_at"), "yyyy-MM-dd"))
        df_em_aberto = df_em_aberto.withColumn("dias_aberto", datediff(lit(hoje.isoformat()), col("created_at")))
        
        df_abandonadas = df_em_aberto.filter(
            (
                ((col("task_type") == "Tarefa a Ser Feita") & (col("dias_aberto") > 15)) |
                ((col("task_type") == "Item de Compra") & (col("dias_aberto") > 30))
            )
        )
        
        count_abandonadas = df_abandonadas.count()
        print(f"\n🎯 RESULTADO PRINCIPAL: {count_abandonadas} TAREFAS ABANDONADAS ENCONTRADAS!")
        
        if count_abandonadas > 0:
            df_abandonadas.select("name", "task_type", "created_at", "dias_aberto").orderBy("dias_aberto", ascending=False).show(50, truncate=False)
            
            # Converter para pandas para relatório final
            df_abandonadas_pd = df_abandonadas.toPandas()
            df_abandonadas_pd['mes_criacao'] = pd.to_datetime(df_abandonadas_pd['created_at']).dt.strftime('%Y-%m')
    else:
        count_abandonadas = 0
        df_abandonadas_pd = pd.DataFrame()

# 9. RELATÓRIO FINAL (sempre usando pandas para eficiência)
print(f"\n9. CRIANDO RELATÓRIO POR MÊS...")

ultimos6_meses = []
for i in range(6):
    mes = (hoje.replace(day=1) - datetime.timedelta(days=30*i))
    ultimos6_meses.append(mes.strftime("%Y-%m"))

print("Meses para relatório:", ultimos6_meses)

# Criar relatório
resultado_final = []

if count_abandonadas > 0 and not df_abandonadas.empty:
    for mes in ultimos6_meses:
        if len(todos_registros) <= 100:
            # Usar dados pandas
            tarefas_mes = df_abandonadas[df_abandonadas['mes_criacao'] == mes]
            count_tarefa_feita = len(tarefas_mes[tarefas_mes['task_type'] == 'Tarefa a Ser Feita'])
            count_item_compra = len(tarefas_mes[tarefas_mes['task_type'] == 'Item de Compra'])
        else:
            # Usar dados pandas convertidos do Spark
            tarefas_mes = df_abandonadas_pd[df_abandonadas_pd['mes_criacao'] == mes]
            count_tarefa_feita = len(tarefas_mes[tarefas_mes['task_type'] == 'Tarefa a Ser Feita'])
            count_item_compra = len(tarefas_mes[tarefas_mes['task_type'] == 'Item de Compra'])
        
        resultado_final.append({
            "mes": mes,
            "Tarefa a Ser Feita": count_tarefa_feita,
            "Item de Compra": count_item_compra
        })
        
        if count_tarefa_feita > 0 or count_item_compra > 0:
            print(f"Mês {mes}: {count_tarefa_feita} tarefas normais, {count_item_compra} itens de compra abandonados")
else:
    # Criar relatório zerado
    for mes in ultimos6_meses:
        resultado_final.append({
            "mes": mes,
            "Tarefa a Ser Feita": 0,
            "Item de Compra": 0
        })

# 10. CRIAR E MOSTRAR RELATÓRIO FINAL
print(f"\n10. RELATÓRIO FINAL...")
df_relatorio = pd.DataFrame(resultado_final)
df_relatorio = df_relatorio.set_index('mes').T
 
# Mostrar tabela no estilo Excel
print("\n=== VISUALIZAÇÃO DA TABELA DE RELATÓRIO ===")
tabela_formatada = tabulate(df_relatorio, headers="keys", tablefmt="grid", stralign="center", numalign="center")
display(Markdown(f"```\n{tabela_formatada}\n```"))
 
print("\n=== RELATÓRIO DE TAREFAS ABANDONADAS (6 MESES) ===")
print(df_relatorio.to_string())
 
# 11. SALVAR RELATÓRIO NA PASTA DATA
print(f"\n11. SALVANDO RELATÓRIO...")
 
# Criar pasta data se não existir
data_folder = "../data"
os.makedirs(data_folder, exist_ok=True)
 
# Caminhos dos arquivos
csv_path = os.path.join(data_folder, "rel_abandono_6meses.csv")
excel_path = os.path.join(data_folder, "rel_abandono_6meses.xlsx")
 
try:
    # CSV primeiro (mais rápido)
    df_relatorio.to_csv(csv_path)
    print(f"✅ Relatório salvo como CSV: {csv_path}")
 
    # Excel depois
    try:
        df_relatorio.to_excel(excel_path)
        print(f"✅ Relatório salvo como Excel: {excel_path}")
    except Exception as e:
        print(f"⚠️ Excel não disponível (CSV salvo): {e}")
 
except Exception as e:
    print(f"❌ Erro ao salvar: {e}")


# 12. RESUMO FINAL OTIMIZADO
print(f"\n=== RESUMO FINAL COMPLETO ===")

if count_abandonadas > 0:
    if len(todos_registros) <= 100:
        # Usar dados pandas
        tipo_counts = df_abandonadas['task_type'].value_counts()
        
        print(f"🎯 TOTAL DE TAREFAS ABANDONADAS: {count_abandonadas}")
        print(f"Breakdown por tipo:")
        for tipo, count in tipo_counts.items():
            print(f"- {tipo}: {count} tarefas abandonadas")
        
        # Lista completa
        print(f"\n=== LISTA COMPLETA DAS TAREFAS ABANDONADAS ===")
        for i, (_, tarefa) in enumerate(df_abandonadas.iterrows(), 1):
            data_formatada = tarefa['created_at'].strftime('%Y-%m-%d') if hasattr(tarefa['created_at'], 'strftime') else str(tarefa['created_at'])
            print(f"{i:2d}. {tarefa['name']} ({tarefa['task_type']}) - {tarefa['dias_aberto']} dias - Criada em {data_formatada}")
    else:
        # Usar dados convertidos do Spark
        print(f"🎯 TOTAL DE TAREFAS ABANDONADAS: {count_abandonadas}")

else:
    print(f"❌ NENHUMA TAREFA ABANDONADA ENCONTRADA")
    print(f"Total de tarefas analisadas: {len(todos_registros)}")
    if 'count_em_aberto' in locals():
        print(f"Tarefas em aberto: {count_em_aberto}")

# Estatísticas finais
print(f"\n=== ESTATÍSTICAS DA ANÁLISE ===")
print(f"- Período analisado: {len(todas_datas)} dias")
print(f"- Queries executadas: {total_queries}")
print(f"- Datas com dados: {len(datas_com_dados)}")
print(f"- Taxa de sucesso: {len(datas_com_dados)/total_queries*100:.1f}%")
print(f"- Total de registros processados: {len(todos_registros)}")
print(f"- Método de processamento: {'Pandas (otimizado)' if len(todos_registros) <= 100 else 'Spark'}")
if 'count_em_aberto' in locals():
    print(f"- Tarefas em aberto analisadas: {count_em_aberto}")
if 'count_abandonadas' in locals():
    print(f"- Tarefas abandonadas encontradas: {count_abandonadas}")

# Finalizar
spark.stop()
print(f"\n✅ Análise concluída com otimizações aplicadas!")

=== ANÁLISE DE TAREFAS ABANDONADAS (OTIMIZADA) ===
Data de referência: 2025-06-06

1. GERANDO DATAS DOS ÚLTIMOS 6 MESES...
Período de análise: 20241208 até 20250606
Total de datas para verificar: 181

2. BUSCANDO DADOS COM QUERY OTIMIZADA...
Processando lote 1: datas 20241208 a 20250126
Processando lote 2: datas 20250127 a 20250317
  Progresso: 3 registros encontrados até agora
Processando lote 3: datas 20250318 a 20250506
  Progresso: 4 registros encontrados até agora
Processando lote 4: datas 20250507 a 20250606
  Progresso: 7 registros encontrados até agora

Resultado da busca:
- Total de queries executadas: 181
- Datas com dados: 7
- Total de registros encontrados: 7
- Primeira data com dados: 20250205
- Última data com dados: 20250529

3. PROCESSANDO DADOS...
📊 Usando processamento Pandas (dataset pequeno)
Colunas disponíveis: ['completed_at', 'task_type', 'user_ID', 'created_at', 'status', 'SK', 'user', 'PK', 'name']
Total de registros: 7

Primeiros 3 registros:
                 

```
+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+
|                    |  2025-06  |  2025-05  |  2025-04  |  2025-03  |  2025-02  |  2025-01  |
+====================+===========+===========+===========+===========+===========+===========+
| Tarefa a Ser Feita |     0     |     1     |     0     |     1     |     1     |     0     |
+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+
|   Item de Compra   |     0     |     0     |     0     |     0     |     0     |     0     |
+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+
```


=== RELATÓRIO DE TAREFAS ABANDONADAS (6 MESES) ===
mes                 2025-06  2025-05  2025-04  2025-03  2025-02  2025-01
Tarefa a Ser Feita        0        1        0        1        1        0
Item de Compra            0        0        0        0        0        0

11. SALVANDO RELATÓRIO...
✅ Relatório salvo como CSV: ../data\rel_abandono_6meses.csv
✅ Relatório salvo como Excel: ../data\rel_abandono_6meses.xlsx

=== RESUMO FINAL COMPLETO ===
🎯 TOTAL DE TAREFAS ABANDONADAS: 3
Breakdown por tipo:
- Tarefa a Ser Feita: 3 tarefas abandonadas

=== LISTA COMPLETA DAS TAREFAS ABANDONADAS ===
 1. Blanditiis corrupti quasi (Tarefa a Ser Feita) - 121 dias - Criada em 2025-02-05
 2. Modi (Tarefa a Ser Feita) - 86 dias - Criada em 2025-03-12
 3. Nobis fugit earum enim quo ipsa (Tarefa a Ser Feita) - 28 dias - Criada em 2025-05-09

=== ESTATÍSTICAS DA ANÁLISE ===
- Período analisado: 181 dias
- Queries executadas: 181
- Datas com dados: 7
- Taxa de sucesso: 3.9%
- Total de registros processad